In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))
trainSet.take(5)

[(2738, 1, 1), (4716, 1, 4), (13298, 1, 8), (15122, 1, 4), (11326, 2, 5)]

In [3]:
mat=CoordinateMatrix(trainSet)

In [4]:
indexedRowfromCoordMat=mat.toIndexedRowMatrix()

In [6]:
cs=indexedRowfromCoordMat.columnSimilarities()
cs.entries.saveAsTextFile("itemSimilarities")

itemSimilarities = sc.textFile("itemSimilarities/")
itemSimilarities.count()

In [21]:
cs1 = cs.entries.filter(lambda x: x.i == 35599)

In [35]:
csrdd = cs.entries.map(lambda x: "{}, {}, {}".format(x.i, x.j, x.value))
csrdd.take(5)

['12368, 26377, 0.34001020045902297',
 '133, 18648, 0.029826529048348538',
 '17975, 36399, 0.14744604466668038',
 '15573, 16221, 0.15482392825253707',
 '32286, 32571, 0.13459547551454137']

In [36]:
prova = csrdd.saveAsTextFile("itemSimil")

In [45]:
text = sc.textFile("itemSimil/")
text = text.map(lambda x: tuple( x.split(','))).map(lambda x: (int(x[0]),int(x[1]),float(x[2])))
highSimilarities = text.filter(lambda x: x[2]==1)
highSimilarities.count()

278834

In [46]:
trainSet2 = trainSet.map(lambda x: (x[1], x[0], x[2]))
trainSet2.take(5)

[(1, 2738, 1), (1, 4716, 4), (1, 13298, 8), (1, 15122, 4), (2, 11326, 5)]

In [48]:
matUser = CoordinateMatrix(trainSet2)
indexedRowMatUser = matUser.toIndexedRowMatrix()
userCs = indexedRowMatUser.columnSimilarities()
userCsRdd = userCs.entries.map(lambda x: "{}, {}, {}".format(x.i, x.j, x.value))
userCsRdd.saveAsTextFile("UserSim")

In [51]:
simUser = sc.textFile("UserSim/")
simUser.count()

1534600

In [59]:
userSimilarities = simUser.map(lambda x: tuple( x.split(','))).map(lambda x: (int(x[0]),int(x[1]),float(x[2])))
sim1 = userSimilarities.filter(lambda x: x[2]!=1)
userSimilaritiesOrdered = userSimilarities.sortBy(lambda x: x[0])
userSimilaritiesOrdered.take(5)

[(1, 6356, 0.06968155546148419),
 (1, 7790, 0.038236904608999804),
 (1, 7932, 0.06170221091733459),
 (1, 950, 0.03712555380137589),
 (1, 6551, 0.030702086603900267)]

In [60]:
userSimilaritiesOrdered.saveAsTextFile("userSimOrdered")